In [35]:
from requests import get
from bs4 import BeautifulSoup
import datetime
import regex as re
import pandas as pd
from unidecode import unidecode
import numpy as np

In [3]:
commentory = pd.read_csv("../../data/commentory.csv")
data_links = pd.read_csv("../../data/cl_data_21_22 _stage_leg.csv")

In [5]:
print(commentory.shape)
print(data_links.shape)

(10159, 14)
(125, 5)


In [6]:
final_df = pd.merge(commentory,data_links,left_on="link",right_on="live_ticker",how = "left")

In [7]:
final_df.shape

(10159, 19)

In [8]:
final_df.head()

,time,comment,event,event_player,event_team,comment_desc,home_team,home_team_abbr,away_team,away_team_abbr,full_time_score,match_x,date,link,match_y,match_day,live_ticker,stage,leg
0,NaN,Thanks for joining our commentary this evenin...,NaN,NaN,NaN,full time summary,Barcelona,BAR,Bayern Munchen,FCB,0 - 3,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Group Stage,1
1,NaN,Barcelona are next in action at home in LaLig...,NaN,NaN,NaN,full time summary,Barcelona,BAR,Bayern Munchen,FCB,0 - 3,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Group Stage,1
2,NaN,Bayern Munich have eased past Barcelona in th...,NaN,NaN,NaN,full time summary,Barcelona,BAR,Bayern Munchen,FCB,0 - 3,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Group Stage,1
3,90 + 2,FULL-TIME: BARCELONA 0-3 BAYERN MUNICH,NaN,NaN,NaN,timer,Barcelona,BAR,Bayern Munchen,FCB,0 - 3,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Group Stage,1
4,90,There will be two minutes of added time.,NaN,NaN,NaN,timer,Barcelona,BAR,Bayern Munchen,FCB,0 - 3,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Group Stage,1


In [32]:
auto_tagging = {'foul' : ['foul'],
                'yellow card' : ['foul'],
                'yellow-card' : ['foul'],
                'free kick in the defensive half' : ['foul'],
                'hand ball' : ['foul','mistake'],
                'red-card' : ['foul','mistake'],
                'red card' : ['foul','mistake'],
                'yellow card for a bad foul' : ['foul','mistake'],
                'block' : ['save','chance'],
                'save' : ['save','chance'],
                'saved' : ['save','chance'],
                "wide" : ['save','chance'],
                'blocekd' : ['save','chance'],
                'close' : ['chance'],
                'woodwork' : ['chance'],
                'offside' : ['information'],
                'corner' : ['block']
                }

In [34]:
tags_list  = list(auto_tagging.keys())
tags_list

['foul',
 'yellow card',
 'yellow-card',
 'free kick in the defensive half',
 'hand ball',
 'red-card',
 'red card',
 'yellow card for a bad foul',
 'block',
 'save',
 'saved',
 'wide',
 'blocekd',
 'close',
 'woodwork',
 'offside',
 'corner']

In [42]:
for tag in tags_list:
    print(tag)
    final_df['tags'] = final_df['comment'].apply(lambda x: auto_tagging[tag] if tag in str.lower(x) else np.NaN)

foul
yellow card
yellow-card
free kick in the defensive half
hand ball
red-card
red card
yellow card for a bad foul
block
save
saved
wide
blocekd
close
woodwork
offside
corner


In [40]:
final_df.head(10)

,time,comment,event,event_player,event_team,comment_desc,home_team,home_team_abbr,away_team,away_team_abbr,full_time_score,match_x,date,link,match_y,match_day,live_ticker,stage,leg,tags
0,NaN,Thanks for joining our commentary this evenin...,NaN,NaN,NaN,full time summary,Barcelona,BAR,Bayern Munchen,FCB,0 - 3,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Group Stage,1,NaN
1,NaN,Barcelona are next in action at home in LaLig...,NaN,NaN,NaN,full time summary,Barcelona,BAR,Bayern Munchen,FCB,0 - 3,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Group Stage,1,NaN
2,NaN,Bayern Munich have eased past Barcelona in th...,NaN,NaN,NaN,full time summary,Barcelona,BAR,Bayern Munchen,FCB,0 - 3,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Group Stage,1,NaN
3,90 + 2,FULL-TIME: BARCELONA 0-3 BAYERN MUNICH,NaN,NaN,NaN,timer,Barcelona,BAR,Bayern Munchen,FCB,0 - 3,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Group Stage,1,NaN
4,90,There will be two minutes of added time.,NaN,NaN,NaN,timer,Barcelona,BAR,Bayern Munchen,FCB,0 - 3,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Group Stage,1,NaN
5,89,CLOSE! Coman bursts down the left flank and c...,NaN,NaN,NaN,timer,Barcelona,BAR,Bayern Munchen,FCB,0 - 3,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Group Stage,1,NaN
6,86,Gnabry goes for a strike from distance after ...,NaN,NaN,NaN,timer,Barcelona,BAR,Bayern Munchen,FCB,0 - 3,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Group Stage,1,NaN
7,85,GOALLLLLLLLLLLLLLLLLL!!!!! LEWANDOWSKI SCORES...,goal,"[' R. Lewandowski ', ' Goal ']",Bayern Munchen,timer,Barcelona,BAR,Bayern Munchen,FCB,0 - 3,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Group Stage,1,NaN
8,84,Gnabry threads a pass into the feet of Sabitz...,NaN,NaN,NaN,timer,Barcelona,BAR,Bayern Munchen,FCB,0 - 3,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Group Stage,1,NaN
9,82,Gavi goes into the book for a foul on Goretzka.,yellow_card,"[' Gavi ', ' Yellow Card ']",Barcelona,timer,Barcelona,BAR,Bayern Munchen,FCB,0 - 3,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,Group Stage,1,NaN


In [44]:
'close' in str.lower('CLOSE! Coman bursts ')

True

In [9]:
filtered_df = final_df[final_df['event'].notnull()]

In [12]:
filtered_df['time_detail'] = filtered_df['time'].apply(lambda x: 'Added Time' if "+" in x else 'Normal Time')

/var/folders/gc/n0p2_5d52qz9v6vm5nmmszx40000gn/T/ipykernel_56764/596250218.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['time_detail'] = filtered_df['time'].apply(lambda x: 'Added Time' if "+" in x else 'Normal Time')


In [20]:
filtered_df['time'] = filtered_df['time'].apply(lambda x: x.replace(" ",""))

/var/folders/gc/n0p2_5d52qz9v6vm5nmmszx40000gn/T/ipykernel_56764/503602325.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['time'] = filtered_df['time'].apply(lambda x: x.replace(" ",""))


In [25]:
filtered_df['time'] = filtered_df['time'].apply(lambda x: (int(re.search("(\d+)\+(\d+)",x).group(1)) + int(re.search("(\d+)\+(\d+)",x).group(2))) if "+" in x else int(x) )

/var/folders/gc/n0p2_5d52qz9v6vm5nmmszx40000gn/T/ipykernel_56764/3439674364.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['time'] = filtered_df['time'].apply(lambda x: (int(re.search("(\d+)\+(\d+)",x).group(1)) + int(re.search("(\d+)\+(\d+)",x).group(2))) if "+" in x else int(x) )


In [28]:
filtered_df = filtered_df.sort_values(by="time",ascending=True)

In [60]:
filtered_df.to_csv("../../data/key_events2.csv",index=False)

In [56]:
df_grouped = filtered_df.groupby(['link','match_x','stage','leg','full_time_score'])['comment'].agg('{'.join)

In [57]:
df_grouped =  df_grouped.to_frame().reset_index()

In [58]:
df_grouped

,link,match_x,stage,leg,full_time_score,comment
0,https://www.goal.com/en/match/ajax-vs-benfica/...,Ajax vs Benfica,R16,2,0 - 1,The visitors have withdrawn Taarabt at the in...
1,https://www.goal.com/en/match/ajax-vs-besiktas...,Ajax vs Besiktas,Group Stage,1,2 - 0,"And it's 1-0 to Ajax! Berghuis, who just miss..."
2,https://www.goal.com/en/match/ajax-vs-borussia...,Ajax vs Borussia Dortmund,Group Stage,1,4 - 0,AJAX GO IN FRONT! A delighted Tadic wheels aw...
3,https://www.goal.com/en/match/ajax-vs-sporting...,Ajax vs Sporting Cp,Group Stage,2,4 - 2,Braganca is also shown a yellow card for the ...
4,https://www.goal.com/en/match/atalanta-vs-manc...,Atalanta vs Manchester United,Group Stage,2,2 - 2,ILICIC SCORES! It's a good cross from Zapata ...
...,...,...,...,...,...,...
120,https://www.goal.com/en/match/young-boys-vs-ma...,Young boys Vs Manchester United,Group Stage,1,2 - 1,Fernandes' ball for that goal was sumptuous. ...
121,https://www.goal.com/en/match/young-boys-vs-vi...,Young boys Vs Villarreal,Group Stage,1,1 - 4,GOOOALLLLLL!!!! Villarreal take the lead!!! I...
122,https://www.goal.com/en/match/zenit-vs-chelsea...,Zenit vs Chelsea,Group Stage,2,3 - 3,GOOOOOAAAAALLLLL!!!! WHAT A START FOR CHELSEA...
123,https://www.goal.com/en/match/zenit-vs-juventu...,Zenit vs Juventus,Group Stage,1,0 - 1,The first yellow card of the game has been du...


In [59]:
df_grouped.to_csv("../../data/key_events.csv",index=False)